In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, f1_score

# Veri setini yükleme 
train_data_dir = r"C:\Users\golde\Desktop\data2\Data\train"
validation_data_dir = r"C:\Users\golde\Desktop\data2\Data\valid"
test_data_dir = r"C:\Users\golde\Desktop\data2\Data\test"

# Veri ön işleme işlemleri için ImageDataGenerator kullanımı
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Verileri yükleme ve ölçeklendirme
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

# CNN Modeli
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Modelin derlenmesi
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
              metrics=['accuracy'])

# Modelin eğitimi
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=50)

# Modelin test edilmesi
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print('Test accuracy:', test_acc)

# Test veri kümesi üzerinde tahmin yapma
y_pred = model.predict(test_generator)

y_pred_binary = np.round(y_pred)

y_true = test_generator.classes
cm = confusion_matrix(y_true, y_pred_binary)
report = classification_report(y_true, y_pred_binary)

# Sensitivity ve Specificity hesaplama
tn, fp, fn, tp = cm.ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

# Accuracy hesaplama
accuracy = (tp + tn) / (tp + tn + fp + fn)

# AUROC hesaplama
auroc = roc_auc_score(y_true, y_pred)

# F1 skoru hesaplama
f1 = f1_score(y_true, y_pred_binary)

# Sonuçları yazdırma
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)
print("Accuracy:", accuracy)
print("AUROC:", auroc)
print("F1 Score:", f1)





In [6]:
# Resimleri tahmin et
import os
from tensorflow.keras.preprocessing import image

test_image = image.load_img(r"C:\Users\golde\Desktop\archive\yyK.jpg", target_size=(150, 150))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
result = model.predict(test_image)

if result[0][0] == 1:
    prediction = 'saglikli'
else:
    prediction = 'kanserli'

print(prediction)


1/1 [==============================] - 0s 39ms/step
kanserli
